In [44]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model

In [45]:
def base_model():
    inputs = Input(shape=(784,), name='clothing')
    x = Dense(64, activation='relu', name='dense_1')(inputs)
    x = Dense(64, activation='relu', name='dense_2')(x)
    output = Dense(10, activation='softmax', name='prediction')(x)
    model = Model(inputs, output)
    return model

In [46]:
import tensorflow_datasets as tfds
train_data = tfds.load('fashion_mnist', split='train');
test_data = tfds.load('fashion_mnist', split='test');

In [47]:
def format_image(data):
    image = data['image']
    image = tf.reshape(image, [-1])
    image = tf.cast(image, tf.float32)
    image = image / 255.0
    return image, data['label']

In [48]:
train_data = train_data.map(format_image)
test_data = test_data.map(format_image)

In [49]:
BATCH_SIZE = 64
train = train_data.shuffle(buffer_size=1024).batch(BATCH_SIZE)
test = test_data.batch(BATCH_SIZE)

In [50]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import SparseCategoricalAccuracy

loss_object = SparseCategoricalCrossentropy()
optimizer = Adam()
train_acc_metrics = SparseCategoricalAccuracy()
val_acc_metrics = SparseCategoricalAccuracy()

In [51]:
def apply_gradient(optimizer, model, x, y):
    with tf.GradientTape() as t:
        logits = model(x)
        loss_value = loss_object(y_true=y, y_pred=logits)
    gradients = t.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))

    return logits, loss_value

In [52]:
def train_data_one_epoch(model, train):
    losses = []
    for step, (x_batch_train, y_batch_train) in enumerate(train):
        logits, loss_value = apply_gradient(optimizer, model, x_batch_train,  y_batch_train)
        losses.append(loss_value)

        train_acc_metrics.update_state(y_batch_train, logits)
    return losses

In [53]:
def perform_validation(model, test):
    losses = []
    for x_val, y_val in test:
        val_logits = model(x_val)
        val_loss = loss_object(y_true=y_val, y_pred=val_logits)
        losses.append(val_loss)

        val_acc_metrics.update_state(y_val, val_logits)
    return losses

In [54]:
import numpy as np
def begin_train(model, epochs, train, test):
    for epoch in epochs:
        losses_train = train_data_one_epoch(model, train)
        train_acc = train_acc_metrics.result()
        train_acc_metrics.reset_states()

        losses_val = perform_validation(model, test)
        val_acc = val_acc_metrics.result()
        val_acc_metrics.reset_states()

        losses_train_mean = np.mean(losses_train)
        losses_val_mean = np.mean(losses_val)

        print("Epoch {}, train loss {}, val loss {}, Train accuracy {}, Validation accuray {}".format(epoch, losses_train_mean, losses_val_mean, train_acc, val_acc))

In [55]:
begin_train(base_model(), range(20), train, test)

Epoch 0, train loss 0.5405731797218323, val loss 0.44456884264945984, Train accuracy 0.8128833174705505, Validation accuray 0.8428999781608582
Epoch 1, train loss 0.3892807364463806, val loss 0.3921355605125427, Train accuracy 0.8613499999046326, Validation accuray 0.8610000014305115
Epoch 2, train loss 0.34948259592056274, val loss 0.3882937729358673, Train accuracy 0.8730000257492065, Validation accuray 0.8622999787330627
Epoch 3, train loss 0.32622265815734863, val loss 0.37648868560791016, Train accuracy 0.8809000253677368, Validation accuray 0.8684999942779541
Epoch 4, train loss 0.31022340059280396, val loss 0.3717998266220093, Train accuracy 0.885283350944519, Validation accuray 0.8740000128746033
Epoch 5, train loss 0.2966555655002594, val loss 0.352491170167923, Train accuracy 0.8899666666984558, Validation accuray 0.8787000179290771
Epoch 6, train loss 0.2831208109855652, val loss 0.3523012399673462, Train accuracy 0.8944166898727417, Validation accuray 0.8776000142097473
Epo